In [19]:
from urllib.request import urlopen  # python自带爬虫库
import json  # python自带的json数据库
from random import randint  # python自带的随机数库
import pandas as pd
pd.set_option('expand_frame_repr', False)  # 当列太多时不换行
pd.set_option('display.max_rows', 5000)  # 最多显示数据的行数

In [20]:
# =====创建随机数的函数
def _random(n=16):
    """
    创建一个n位的随机整数
    :param n:
    :return:
    """
    start = 10**(n-1)
    end = (10**n)-1
 
    return str(randint(start, end))

In [21]:
# =====获取日、周、月的K线数据
# ===神奇的网址
# 获取K线数据：http://web.ifzq.gtimg.cn/appstock/app/fqkline/get?_var=kline_dayqfq¶m=sz000001,day,,,50,qfq&r=0.5643184591626897
# 正常网址：http://stockhtm.finance.qq.com/sstock/ggcx/000001.shtml

In [29]:
# ===构建网址
# 参数
stock_code = 'sz000001'  # 正常股票sz000001，指数sh000001, ETF sh510500
k_type = 'day'  # day, week, month分别对用日线、周线、月线
num = 10  # 股票最多不能超过640，指数、etf等没有限制

In [30]:
# 构建url
url = 'http://web.ifzq.gtimg.cn/appstock/app/fqkline/get?_var=kline_%sqfq¶m=%s,%s,,,%s,qfq&r=0.%s'
url = url % (k_type, stock_code, k_type, num, _random())
url

'http://web.ifzq.gtimg.cn/appstock/app/fqkline/get?_var=kline_dayqfq¶m=sz000001,day,,,10,qfq&r=0.2196835820682738'

In [31]:
# ===获取数据
content = urlopen(url).read().decode()  # 使用python自带的库，从网络上获取信息


UnicodeEncodeError: 'ascii' codec can't encode character '\xb6' in position 47: ordinal not in range(128)

In [ ]:
# ===将数据转换成dict格式
content = content.split('=', maxsplit=1)[-1] # 分割第一个出现的等号，取后半部分
content = json.loads(content)  # 把浏览器中的json的字符串格式转换我们字典格式

In [ ]:
# ===将数据转换成DataFrame格式
k_data = content['data'][stock_code]
if k_type in k_data: # 对k_type做一个判断，data,stock_code,k_type都是这个字典中的建，选定这些键来确定它的值
    k_data = k_data[k_type]
elif 'qfq' + k_type in k_data:  # qfq是前复权的缩写
    k_data = k_data['qfq' + k_type]
else:
    raise ValueError('已知的key在dict中均不存在，请检查数据')
df = pd.DataFrame(k_data)

In [ ]:
# ===对数据进行整理
rename_dict = {0: 'candle_end_time', 1: 'open', 2: 'close', 3: 'high', 4: 'low', 5: 'amount', 6: 'info'}
# 其中amount单位是手，说明数据不够精确
df.rename(columns=rename_dict, inplace=True)
df['candle_end_time'] = pd.to_datetime(df['candle_end_time'])
if 'info' not in df:
    df['info'] = None
df = df[['candle_end_time', 'open', 'high', 'low', 'close', 'amount', 'info']]
print(df)

In [ ]:
df.to_csv('sh000001.csv', index=False)